In [1]:
import os
import json
import requests
import collections.abc
from azureml.core import Workspace, ComputeTarget
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from dotenv import load_dotenv
from azureml.core.compute import AksCompute
from azureml.core.webservice import AksWebservice



In [ ]:
# Load the environment variables from .env file
load_dotenv()

# Now you can retrieve each variable using os.getenv('VARIABLE_NAME')
subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
workspace_name = os.getenv('WORKSPACE_NAME')
region = os.getenv('REGION')

# You can now use these variables in your application
print(subscription_id, resource_group, workspace_name, region)


In [4]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying AppInsights with name testfrominsights2b321920.
Deployed AppInsights with name testfrominsights2b321920. Took 3.67 seconds.
Deploying StorageAccount with name testfromstoragea76862e2c.
Deploying KeyVault with name testfromkeyvault46a7a01b.
Deployed KeyVault with name testfromkeyvault46a7a01b. Took 21.32 seconds.
Deploying Workspace with name TestFromHome-1.
Deployed StorageAccount with name testfromstoragea76862e2c. Took 27.3 seconds.
Deployed Workspace with name TestFromHome-1. Took 35.67 seconds.
Workspace TestFromHome-1 created


In [5]:
# Specify the path to your SavedModel directory
model_path = 'sentiment_analysis_model'
model_name = 'sentiment_analysis_model'

# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path,
                                  model_name=model_name,
                                  workspace=ws)

print("Model registered successfully:", registered_model.id)


Registering model sentiment_analysis_model
Model registered successfully: sentiment_analysis_model:1


In [6]:
# Specify the local file path to the tokenizer
tokenizer_file_path = 'my_tokenizer/tokenizer.pickle'  # Update this path

# Specify a name for the tokenizer model
tokenizer_model_name = 'my_tokenizer'

# Register the tokenizer as a model in Azure ML
registered_tokenizer = Model.register(model_path=tokenizer_file_path,
                                      model_name=tokenizer_model_name,
                                      workspace=ws)

print("Tokenizer model registered successfully:", registered_tokenizer.id)


Registering model my_tokenizer
Tokenizer model registered successfully: my_tokenizer:1


In [7]:
# Create a new environment
conda_env = Environment('my-tf-conda-env-home-1')
# Define the conda dependencies object
conda_deps = CondaDependencies()
# Add conda and pip packages
# Note: Specify the versions if necessary to ensure compatibility
conda_deps.add_conda_package('numpy')
conda_deps.add_pip_package('tensorflow')  # For TensorFlow

# Set the dependencies for the environment
conda_env.python.conda_dependencies = conda_deps

In [19]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='script.py', environment=conda_env)

In [20]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [21]:
# Assuming 'registered_model' is your main model and 'registered_tokenizer' is your tokenizer model
service = Model.deploy(workspace=ws,
                       name='sentiment-home-3',  # Name of the deployment
                       models=[registered_model, registered_tokenizer],  # Include both models
                       inference_config=inference_config,  # Your inference configuration
                       deployment_config=aci_config)  # Your deployment configuration
service.wait_for_deployment(show_output=True)


C:\Users\ALITEMP\AppData\Local\Temp\ipykernel_21540\2281033007.py:2: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-12 19:54:09+03:00 Creating Container Registry if not exists.
2024-02-12 19:54:09+03:00 Registering the environment.
2024-02-12 19:54:12+03:00 Use the existing image.
2024-02-12 19:54:12+03:00 Generating deployment configuration.
2024-02-12 19:54:14+03:00 Submitting deployment to compute.
2024-02-12 19:54:19+03:00 Checking the status of deployment sentiment-home-3..
2024-02-12 19:56:13+03:00 Checking the status of inference endpoint sentiment-home-3.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
#service.get_logs()

In [22]:
scoring_uri = service.scoring_uri


In [23]:
scoring_uri


'http://a0aafa14-0bf4-4c77-9123-9b998a0eb35d.uaenorth.azurecontainer.io/score'